# T1546.007 - Netsh Helper DLL
Adversaries may establish persistence by executing malicious content triggered by Netsh Helper DLLs. Netsh.exe (also referred to as Netshell) is a command-line scripting utility used to interact with the network configuration of a system. It contains functionality to add helper DLLs for extending functionality of the utility.(Citation: TechNet Netsh) The paths to registered netsh.exe helper DLLs are entered into the Windows Registry at <code>HKLM\SOFTWARE\Microsoft\Netsh</code>.

Adversaries can use netsh.exe helper DLLs to trigger execution of arbitrary code in a persistent manner. This execution would take place anytime netsh.exe is executed, which could happen automatically, with another persistence technique, or if other software (ex: VPN) is present on the system that executes netsh.exe as part of its normal functionality.(Citation: Github Netsh Helper CS Beacon)(Citation: Demaske Netsh Persistence)

## Atomic Tests

### Atomic Test #1 - Netsh Helper DLL Registration
You can register a "helper dll" with Netsh as a persistance mechanism. The code in the dll is executed every time netsh.exe is called.
The NetshHelper.dll provided with the atomic will simply launch notepad when netsh.exe is run.

[Blog](https://htmlpreview.github.io/?https://github.com/MatthewDemaske/blogbackup/blob/master/netshell.html)
[Sample DLL code](https://github.com/outflanknl/NetshHelperBeacon)

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Helper DLL must exist on disk at specified location (#{helper_file})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1546.007\bin\NetshHelper.dll") { exit 0} else { exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path PathToAtomicsFolder\T1546.007\bin\NetshHelper.dll) -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1546.007/bin/NetshHelper.dll" -OutFile "PathToAtomicsFolder\T1546.007\bin\NetshHelper.dll"

```

In [ ]:
Invoke-AtomicTest T1546.007 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
netsh.exe add helper PathToAtomicsFolder\T1546.007\bin\NetshHelper.dll
taskkill /im notepad.exe /t /f > NUL 2>&1
```

In [ ]:
Invoke-AtomicTest T1546.007 -TestNumbers 1

#### Cleanup: 
```cmd
netsh.exe delete helper PathToAtomicsFolder\T1546.007\bin\NetshHelper.dll
```

In [ ]:
Invoke-AtomicTest T1546.007 -TestNumbers 1 -Cleanup

## Detection
It is likely unusual for netsh.exe to have any child processes in most environments. Monitor process executions and investigate any child processes spawned by netsh.exe for malicious behavior. Monitor the <code>HKLM\SOFTWARE\Microsoft\Netsh</code> registry key for any new or suspicious entries that do not correlate with known system files or benign software.(Citation: Demaske Netsh Persistence)